## MGMTMSA 403 Homework 2: Portfolio Optimization


## Pre-processing step: Estimate expected returns and covariance

In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv
import pandas as pd

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickets of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## collect index that corresponds to each ticker
    #print(tick4 , tickind )
# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))


##  Model 1

In [2]:
#create the model and add decision variables
mod = Model()
d_Q1 = 4
wi = mod.addVars(d_Q1)

Academic license - for non-commercial use only


In [3]:
#adding constraints on weight for the 4 tickers
mod.addConstr(sum(wi[i] for i in range(len(tickind)))==1)
for i in range(len(tickind)):
    mod.addConstr(wi[i]>=0)
mod.addConstr(sum(wi[i]*avg_return[tickind[i]] for i in range(len(tickind)))>=0.005)


<gurobi.Constr *Awaiting Model Update*>

In [4]:
#set the objective function to minimize risk
mod.setObjective(sum((wi[i]*wi[j]*C[tickind[i],tickind[j]]) 
                for j in range (len(tickind)) for i in range(len(tickind))), GRB.MINIMIZE)

In [5]:
mod.update()

In [6]:
mod.optimize()

Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.02s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.87864836e+05 -1.87864836e+05  3.50e+03 1.91e-07  1.00e+06     0s
   1   5.03195868e+03 -5.16248579e+03  2.59e+02 1.41e-08  7.60e+04     0s
   2   8.65509708e-03 -1.63980611e+02  5.37e-01 2.93e-11  1.91e+02     0s
   3   9.86310203e-04 -1.1168577

In [7]:
#for i in range(len(tickind)):
#    print(wi[i].x)

##  1a) 
For Model 1, write down the optimal risk (i.e. the optimal objective function value), solver
time, and the weight on each of the four stocks.

Answer: Optimal Risk: 1.77493265e-04  
        Solver Time : 0.05
        secs  
        Weight of the 4 stocks:   
        MSFT: 0.23711749137103624  
        GS: 0.02585983533067896  
        PG: 1.574971013220768e-09  
        SCHP: 0.7370226717232814  
        
        

##  Model 2

In [8]:
#setting decision variables for model 2
d_Q2 = 390

wi2 = mod.addVars(d_Q2)

In [9]:
#setting constraints for model 2
mod.addConstr(sum(wi2[i] for i in range(len(C)))==1)
for i in range(len(C)):
    mod.addConstr(wi2[i]>=0)
mod.addConstr(sum(wi2[i]*avg_return[i] for i in range(len(C)))>=0.005)



<gurobi.Constr *Awaiting Model Update*>

In [10]:
#setting the objective function to minimize risk for all 390 tickers
mod.setObjective(sum((wi2[i]*wi2[j]*C[i,j]) 
                     for j in range (len(C)) for i in range(len(C))), GRB.MINIMIZE)

In [11]:
mod.update()
mod.optimize()

Optimize a model with 398 rows, 394 columns and 1182 nonzeros
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 394 rows and 0 columns
Presolve time: 0.03s
Presolved: 4 rows, 394 columns, 788 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.831e+03
 Factor NZ  : 1.894e+03
 Factor Ops : 7.754e+04 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.89821559e-13 -2.89821559e-13  3.90e+05 3.22e-07  1.00e+06     0s
   1   2.49696916e+03 -4.51696832e+03  1.43e+04 1.18e-08  3.69e+04     0s
   2   3.77059123e-03 -2.10177389e+03  1.35e+01 1.12e-11  4.00e+01     0s
   3   9.4513

## 1b) 
For Model 2, write down the optimal risk and solver time:  
    Optimal Risk: 2.87887482e-05  
    Solver Time: 0.10 seconds 

##  Model 3

In [12]:
#Setting the decision Variables for Model 3
d_Q3 = 390
wi3 = mod.addVars(d_Q3)
x = mod.addVars(d_Q3, vtype = GRB.BINARY)

In [13]:
#Defining the constraints for model 3
mod.addConstr(sum(wi3[i] for i in range(len(C)))==1)
for i in range(len(C)):
    mod.addConstr(wi3[i]>=0)
mod.addConstr(sum(wi3[i]*avg_return[i] for i in range(len(C)))>=0.005)
for i in range(len(C)):
    mod.addConstr(x[i] >= wi3[i])
mod.addConstr(sum(x[i] for i in range(len(C))) <=4)

<gurobi.Constr *Awaiting Model Update*>

In [14]:
#setting the objective function to minimize risk for at most 4 tickers
mod.setObjective(sum((wi3[i]*wi3[j]*C[i,j]) 
                     for j in range (len(C)) for i in range(len(C))), GRB.MINIMIZE)

In [15]:
mod.update()

In [16]:
mod.optimize()

Optimize a model with 1181 rows, 1174 columns and 3522 nonzeros
Model has 76245 quadratic objective terms
Variable types: 784 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolve removed 788 rows and 394 columns
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00003    0   21    0.00090    0.00003  96.8%     -    0s
H    0     0                       0.0000783    0.00003  63.3%     -    0s
    

## 1 c)  
For Model 3, report the optimal risk, solver time, and the ticker and weight on each of the
four stocks selected by the model.  

Optimal Risk: 6.753470760728e-05  
Solver Time: 109.53 seconds  
Ticker and Weight: See code below:


In [17]:
#weight of the 4 stocks selected by the model
for i in range(len(C)):
    if x[i].x ==1:
        print(tickers[i], wi3[i].x)

CME 0.12641141929490476
LLY 0.07547619035437456
NVDA 0.0437537067284318
BND 0.7543586836222889


##  Model 3 (with parameters) 

In [18]:
#Setting decision variables
d_Q4 = 390
wi4 = mod.addVars(d_Q4)
x = mod.addVars(d_Q4, vtype = GRB.BINARY)

In [19]:
#using constraints from model 3
mod.addConstr(sum(wi4[i] for i in range(len(C)))==1)
for i in range(len(C)):
    mod.addConstr(wi4[i]>=0)
mod.addConstr(sum(wi4[i]*avg_return[i] for i in range(len(C)))>=0.005)
for i in range(len(C)):
    mod.addConstr(x[i] >= wi4[i])
mod.addConstr(sum(x[i] for i in range(len(C))) <=4)

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
#setting the objective value
mod.setObjective(sum((wi4[i]*wi4[j]*C[i,j]) 
                     for j in range (len(C)) for i in range(len(C))), GRB.MINIMIZE)

In [21]:
#setting Gurobi to timeout
mod.Params.TimeLimit = 30.0

Changed value of parameter TimeLimit to 30.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100


In [22]:
mod.update()

In [23]:
mod.optimize()

Optimize a model with 1964 rows, 1954 columns and 5862 nonzeros
Model has 76245 quadratic objective terms
Variable types: 1174 continuous, 780 integer (780 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]

MIP start produced solution with objective 0.00118466 (0.08s)
MIP start produced solution with objective 0.000764814 (0.09s)
MIP start produced solution with objective 8.32013e-05 (0.13s)
MIP start produced solution with objective 8.18709e-05 (0.51s)
MIP start produced solution with objective 7.06604e-05 (1.27s)
MIP start produced solution with objective 6.75857e-05 (1.32s)
MIP start produced solution with objective 6.75347e-05 (1.89s)
Loaded MIP start with objective 6.75347e-05
Processed MIP start in 3.56 seconds

Presolve removed 1178 rows and 394 columns
Presolve time: 0.03s
Presolved: 786 rows, 1560 columns, 3900 nonzeros
Presolv

## 3a)

Set Gurobi to terminate after 30 seconds by including XYZ.Params.TimeLimit = 30.0 in
your code for Model 3, where ’XYZ’ is the name of your model. How does the objective
function value at termination compare the optimal value obtained in question 1c)?

Answer: The best objective value does not change. However, due to temporal constraints, Gurobi terminates  at a gap of 20% from the objective value, with the best bound value being lower than in 1c).

In [24]:
#setting Gurobi to terminate at a gap of 10%
mod.Params.MIPGap = 0.1

Changed value of parameter MIPGap to 0.1
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001


In [25]:
mod.update()

In [26]:
mod.optimize()

Optimize a model with 1964 rows, 1954 columns and 5862 nonzeros
Model has 76245 quadratic objective terms
Variable types: 1174 continuous, 780 integer (780 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolved: 786 rows, 1560 columns, 3900 nonzeros
Presolved model has 76245 quadratic objective terms

Continuing optimization...

 29201 11681    0.00005   28   22    0.00007    0.00005  20.5%  33.6   30s
 34563 11725    0.00006   41   27    0.00007    0.00006  15.5%  33.2   35s
 37314 11715     cutoff   48         0.00007    0.00006  14.3%  33.1   40s
 42948 11415    0.00006   32   21    0.00007    0.00006  12.4%  32.8   45s
 47849 11454    0.00006   61   18    0.00007    0.00006  11.3%  32.1   50s
 53541 11737     cutoff   45         0.00007    0.00006  10.2%  32.0   55s

Cutting planes:
  MIR: 5
  Flow cover: 6

Explored 55786 node

## 3b)
 Set Gurobi to terminate after reaching a gap of 10% by including XYZ.Params.MIPGap = 0.1
in your code for Model 3, where ’XYZ’ is the name of your model. (Note: The default gap
in Gurobi is 0.0001.) How does the solver time compare with the solution time obtained in
question 1c)?  
  
  Answer: The optimal value stays the same, but since we set Gurobi to terminate within 10% gap, the gap value is not 0.0% as in 1c) which means that the solver, while tracing the path to the optimal value terminated within the 10% bounds of it.

## 2 a) 
Is the optimal risk in Model 1 higher or lower than Model 2? Explain why in 1-2 sentences.  
Optimal Risk in Model 1 is higher than in Model 2 because there are more degrees of freedom, more nodes to explore in terms of achieving an optimal minimum in model 2 (In comparison to 390 tickers for model 2, model 1 deals only with 4 tickers). 

##  2b)
Is the optimal risk in Model 2 higher or lower than Model 3? Explain why in 1-2 sentences.  
  
Optimal Risk in Model 2 is the least of all 3 because the model has unresticted freedom to approach the optimal solution. In model 3, this is restricted again by picking only the 4 most optimal ticker values. While not as arbitrary as in model 1, this is still not as free as in model 2, and thus, the Model 2 has the lowest optimal risk.
  
